In [7]:
!pip install -U "sentence-transformers[train]"

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00


# Imports

In [1]:
import json
import numpy as np
from torch.utils.data import DataLoader, SubsetRandomSampler

from Dataset import SquadDataset
from Model import TrainModel


from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)
from sentence_transformers.losses import CoSENTLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

# Set the train/test data

In [2]:
# import datasets and set train_model
dataset = SquadDataset('data/train-v2.0.json', 'data/test-v2.0.json', train_size=10000, test_size=1000)

train_data = dataset._train_data.shuffle()
test_data = dataset._test_data.shuffle()

In [3]:
# rename columns to match training params
train_data = train_data.rename_columns({'questions_and_contexts': 'sentences1', 'sentences': 'sentences2', 'labels' : 'score'})
test_data = test_data.rename_columns({'questions_and_contexts': 'sentences1', 'sentences': 'sentences2', 'labels' : 'score'})

# Training

In [4]:
# declare the training model
train_model = TrainModel(3, train_data, test_data, save_path = 'models')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# train and save the model
train_model._train()
train_model._save_model()

EmbeddingSimilarityEvaluator : base-model
Cosine-Similarity      : Pearson: 0.0018 Spearman: 0.1771
Manhattan-Distance     : Pearson: 0.0486 Spearman: 0.1788
Euclidean-Distance     : Pearson: 0.0577 Spearman: 0.1771
Dot-Product-Similarity : Pearson: 0.0018 Spearman: 0.1771


Step,Training Loss
500,4.460000
1000,4.103300
1500,4.103100


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

EmbeddingSimilarityEvaluator : trained-model
Cosine-Similarity      : Pearson: -0.2082 Spearman: -0.0503
Manhattan-Distance     : Pearson: -0.1677 Spearman: -0.0486
Euclidean-Distance     : Pearson: -0.1680 Spearman: -0.0488
Dot-Product-Similarity : Pearson: -0.2082 Spearman: -0.0508
